In [1]:
import numpy as np
import pandas as pd
import re

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

from bokeh.io import curdoc, push_notebook, output_notebook
from bokeh.layouts import column, layout
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput, HoverTool
from bokeh.plotting import figure, show
from ipywidgets import interact, interactive, fixed, interact_manual


In [2]:
data = pd.read_csv('data\ocosmetic_p.csv')
data

,Label,name,price,brand,ingredients
0,Moisturizer,Crème de la Mer,175,LA MER,"['capric triglyceride', 'cetyl alcohol', 'prop..."
1,Moisturizer,Facial Treatment Essence,179,SK-II,"['homosalate', 'glycerin', 'octocrylene', 'eth..."
2,Moisturizer,Protini™ Polypeptide Cream,68,DRUNK ELEPHANT,"['sodium hyaluronate', 'sodium hyaluronate', '..."
3,Moisturizer,The Moisturizing Soft Cream,175,LA MER,"['ammonium lactate', 'c12-15', 'glycerin', 'pr..."
4,Moisturizer,Your Skin But Better™ CC+™ Cream with SPF 50+,38,IT COSMETICS,"['glycerin', 'cetearyl alcohol', 'capric trigl..."
...,...,...,...,...,...
1467,Sun protect,Yoghurt Nourishing Fluid Veil Face Sunscreen B...,35,KORRES,"['cocoglycerides', 'ethylhexyl palmitate', 'is..."
1468,Sun protect,Daily Deflector™ Waterlight Broad Spectrum SPF...,48,KATE SOMERVILLE,"['oregano', 'citronella', 'melissa', 'cymbopog..."
1469,Sun protect,Self Tan Dry Oil SPF 50,54,VITA LIBERATA,"['prunus amygdalus dulcis', 'prunus armeniaca ..."
1470,Sun protect,Pro Light Self Tan Bronzing Mist,20,ST. TROPEZ TANNING ESSENTIALS,"['prunus amygdalus dulcis', 'vitis vinifera ex..."


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Label        1472 non-null   object
 1   name         1472 non-null   object
 2   price        1472 non-null   int64 
 3   brand        1472 non-null   object
 4   ingredients  1472 non-null   object
dtypes: int64(1), object(4)
memory usage: 34.6+ KB


In [4]:
for i in range(len(data['ingredients'])):
    data['ingredients'].iloc[i] = str(data['ingredients'].iloc[i]).replace('[', '').replace(']', '').replace("'", '').replace('"', '')

C:\Users\HP\miniconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
all_ingreds = []

for i in data['ingredients']:
    ingreds_list = i.split(', ')
    for j in ingreds_list:
        all_ingreds.append(j)

In [6]:
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['',
 '1,10-decanediol',
 '1,2-hexanediol',
 '1,2-hexanediol ',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract']

In [7]:
all_ingreds.remove('')
for i in range(len(all_ingreds)):
    if all_ingreds[i][-1] == ' ':
        all_ingreds[i] = all_ingreds[i][0:-1]
        
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['1,10-decanediol',
 '1,2-hexanediol',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract',
 'acer saccharum extract',
 'acetate']

In [8]:
one_hot_list = [[0] * 0 for i in range(len(all_ingreds))]

for i in data['ingredients']:
    k=0
    for j in all_ingreds:
        if j in i:
            one_hot_list[k].append(1)
        else:
            one_hot_list[k].append(0)
        k+=1
        
ingred_matrix = pd.DataFrame(one_hot_list).transpose()
ingred_matrix.columns = [sorted(set(all_ingreds))]

ingred_matrix

,"1,10-decanediol","1,2-hexanediol",1-methylhydantoin-2-imide,10-hydroxydecanoic acid,"2,6-dimethyl-7-octen-2-ol","2-bromo-2-nitropropane-1,3-diol",2-oleamido-1,3-o-ethyl ascorbic acid,3-octadecanediol,4-t-butylcyclohexanol,...,zinc glycine,zinc laurate,zinc oxide,zinc pca,zinc sulfate,zingiber aromaticus extract,zingiber cassumunar root oil,zingiber officinale root extract,zingiber officinale root oil,zizyphus jujuba seed extract
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
svd = TruncatedSVD(n_components=150, n_iter = 1000, random_state = 6) # firstly reduce features to 150 with truncatedSVD - this suppresses some noise
svd_features = svd.fit_transform(ingred_matrix)
tsne = TSNE(n_components = 2, n_iter = 1000000, random_state = 6) # reduce 150 features to 2 using t-SNE with exact method
tsne_features = tsne.fit_transform(svd_features)

data['X'] = tsne_features[:, 0]
data['Y'] = tsne_features[:, 1]

In [10]:
unique_types = ['Cleanser','Eye cream','Face Mask','Moisturizer','Sun protect','Treatment']

source = ColumnDataSource(data)

plot = figure(title = "Mapped Similarities", width = 800, height = 600)
plot.xaxis.axis_label = "t-SNE 1"
plot.yaxis.axis_label = 't-SNE 2'

plot.circle(x = 'X', y = 'Y', source = source, fill_alpha=0.7, size=10,
           color = '#000000', alpha = 1)

plot.background_fill_color = "#FFFFFF"
plot.background_fill_alpha = 0.3

hover = HoverTool(tooltips=[('Product', '@name'), ('Brand', '@brand'), ('Price', '@price')])
plot.add_tools(hover)

def type_updater(product_type = unique_types[0]):
    new_data = {'X' : data[data['Label'] == product_type]['X'],
                'Y' : data[data['Label'] == product_type]['Y'],
                'name' : data[data['Label'] == product_type]['name'],
                'brand' : data[data['Label'] == product_type]['brand'],
                'price' : data[data['Label'] == product_type]['price']}
    source.data = new_data
    push_notebook()

In [19]:
output_notebook()
show(plot, notebook_handle = True)
interact(type_updater, product_type = unique_types)

Loading BokehJS ...

interactive(children=(Dropdown(description='product_type', options=('Cleanser', 'Eye cream', 'Face Mask', 'Moi…

<function __main__.type_updater(product_type='Cleanser')>

In [12]:
data

,Label,name,price,brand,ingredients,X,Y
0,Moisturizer,Crème de la Mer,175,LA MER,"capric triglyceride, cetyl alcohol, propanedio...",-41.436417,35.650265
1,Moisturizer,Facial Treatment Essence,179,SK-II,"homosalate, glycerin, octocrylene, ethylhexyl,...",0.738286,-61.757298
2,Moisturizer,Protini™ Polypeptide Cream,68,DRUNK ELEPHANT,"sodium hyaluronate, sodium hyaluronate, panthe...",-40.228352,11.734594
3,Moisturizer,The Moisturizing Soft Cream,175,LA MER,"ammonium lactate, c12-15, glycerin, prunus amy...",8.909954,-37.910595
4,Moisturizer,Your Skin But Better™ CC+™ Cream with SPF 50+,38,IT COSMETICS,"glycerin, cetearyl alcohol, capric triglycerid...",2.640046,-66.719543
...,...,...,...,...,...,...,...
1467,Sun protect,Yoghurt Nourishing Fluid Veil Face Sunscreen B...,35,KORRES,"cocoglycerides, ethylhexyl palmitate, isopropy...",-20.257784,53.518463
1468,Sun protect,Daily Deflector™ Waterlight Broad Spectrum SPF...,48,KATE SOMERVILLE,"oregano, citronella, melissa, cymbopogon schoe...",13.956185,17.455460
1469,Sun protect,Self Tan Dry Oil SPF 50,54,VITA LIBERATA,"prunus amygdalus dulcis, prunus armeniaca frui...",49.426323,20.765572
1470,Sun protect,Pro Light Self Tan Bronzing Mist,20,ST. TROPEZ TANNING ESSENTIALS,"prunus amygdalus dulcis, vitis vinifera extrac...",40.619499,21.338074


In [13]:
sorted(data.brand.unique())

['ALGENIST',
 'AMOREPACIFIC',
 'ANTHONY',
 'APIVITA',
 'BAREMINERALS',
 'BEAUTYBIO',
 'BECCA',
 'BELIF',
 'BIOEFFECT',
 'BIOSSANCE',
 'BLACK UP',
 'BLITHE',
 'BOBBI BROWN',
 'BOSCIA',
 'CANE + AUSTIN',
 'CAUDALIE',
 'CHARLOTTE TILBURY',
 'CLARINS',
 'CLARISONIC',
 'CLINIQUE',
 'COOLA',
 'COVER FX',
 'DERMADOCTOR',
 'DERMAFLASH',
 'DIOR',
 'DR ROEBUCK’S',
 'DR. BRANDT SKINCARE',
 'DR. DENNIS GROSS SKINCARE',
 'DR. JART+',
 'DRUNK ELEPHANT',
 'EDIBLE BEAUTY',
 'ERBORIAN',
 'ERNO LASZLO',
 'ESTÉE LAUDER',
 'EVE LOM',
 'FARMACY',
 'FARSÁLI',
 'FENTY BEAUTY BY RIHANNA',
 'FIRST AID BEAUTY',
 'FOREO',
 'FRESH',
 'GLAMGLOW',
 'GLOW RECIPE',
 'GO-TO',
 'GUERLAIN',
 'HERBIVORE',
 'HUM NUTRITION',
 'ILIA',
 'INDIE LEE',
 'IT COSMETICS',
 'J.ONE',
 'JACK BLACK',
 'JOSIE MARAN',
 'KAPLAN MD',
 'KAT VON D',
 'KATE SOMERVILLE',
 'KENZOKI',
 "KIEHL'S SINCE 1851",
 'KLORANE',
 'KOH GEN DO',
 'KOPARI',
 'KORA ORGANICS',
 'KORRES',
 "L'OCCITANE",
 'LA MER',
 'LANCER',
 'LANCÔME',
 'LANEIGE',
 'LAURA MER

In [14]:
def recommender(search):
    cs_list = []
    brands = []
    output = []
    binary_list = []
    idx = data[data['name'] == search].index.item()
    for i in ingred_matrix.iloc[idx][1:]:
        binary_list.append(i)    
    point1 = np.array(binary_list).reshape(1, -1)
    point1 = [val for sublist in point1 for val in sublist]
    prod_type = data['Label'][data['name'] == search].iat[0]
    brand_search = data['brand'][data['name'] == search].iat[0]
    data_by_type = data[data['Label'] == prod_type]
    for j in range(data_by_type.index[0], data_by_type.index[0] + len(data_by_type)):
        binary_list2 = []
        for k in ingred_matrix.iloc[j][1:]:
            binary_list2.append(k)
        point2 = np.array(binary_list2).reshape(1, -1)
        point2 = [val for sublist in point2 for val in sublist]
        dot_product = np.dot(point1, point2)
        norm_1 = np.linalg.norm(point1)
        norm_2 = np.linalg.norm(point2)
        cos_sim = dot_product / (norm_1 * norm_2)
        cs_list.append(cos_sim)
    data_by_type = pd.DataFrame(data_by_type)
    data_by_type['cos_sim'] = cs_list
    data_by_type = data_by_type.sort_values('cos_sim', ascending=False)
    data_by_type = data_by_type[data_by_type.name != search] 
    l = 0
    for m in range(len(data_by_type)):
        brand = data_by_type['brand'].iloc[l]
        if len(brands) == 0:
            if brand != brand_search:
                brands.append(brand)
                output.append(data_by_type.iloc[l])
        elif brands.count(brand) < 2:
            if brand != brand_search:
                brands.append(brand)
                output.append(data_by_type.iloc[l])
        l += 1
        
    return print('\033[1m', 'Products similar to', search,':', '\033[0m'), print(pd.DataFrame(output)[['name', 'cos_sim']].head(5))

In [15]:
recommender("Facial Treatment Essence")

 Products similar to Facial Treatment Essence : 
                                         name   cos_sim
5                             The Water Cream  0.635411
8                          Ultra Facial Cream  0.544248
100  Cold Plasma Sub-D Firming Neck Treatment  0.537225
40                    Luminous Dewy Skin Mist  0.533745
6                   Lala Retro™ Whipped Cream  0.520401


(None, None)

In [21]:
recommender("The Water Cream")

 Products similar to The Water Cream : 
                                             name   cos_sim
6                       Lala Retro™ Whipped Cream  0.748355
10               Lotus Youth Preserve Moisturizer  0.736570
11                  Midnight Recovery Concentrate  0.736570
20                           Crème de la Mer Mini  0.736570
4   Your Skin But Better™ CC+™ Cream with SPF 50+  0.736570


(None, None)